Merge all

In [41]:
import pandas as pd

# # H1N1pdm HA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/ha/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/ha/positions_h1n1pdm_HA1.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/ha/glycosylation_sites_HA1.csv", sep=",")

# # H1N1pdm NA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/na/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/na/positions_h1n1pdm_NA.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/na/glycosylation_sites_NA.csv", sep=",")

# H1N1pdm PA data
df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/pa/nextclade.csv", sep=";")
df_positions = pd.read_csv("../../output/positions/h1n1pdm/pa/positions_h1n1pdm_PA.csv", sep=",")
df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/pa/glycosylation_sites_PA.csv", sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_positions = df_positions.reset_index(drop=True)
df_glycosylation = df_glycosylation.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

# Print the final dataframes to be used.
# print(f"This is 'df_nextclade' output: \n {df_nextclade.head()}")
# print(f"This is 'df_positions' output: \n {df_positions.head()}")
# print(f"This is 'df_glycosylation' output: \n {df_glycosylation.head()}")


# Merge all three tables into one Dataframe 'df'.
df = df_positions.merge(df_glycosylation, how="left", on="seqName") \
                .merge(df_nextclade, how="left", on="seqName")


# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Reorder the DataFrame columns
df = df[final_columns]

print(df)


# HA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/ha/merged_tryout_all_ha.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/na/merged_tryout_all_na.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
df.to_csv("../../output/positions/h1n1pdm/pa/merged_tryout_all_pa.csv", index=True)



                                                seqName  clade 198 199   222  \
0     A/Netherlands/10452/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E  NFSC   
1     A/Netherlands/00087/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E  NFSC   
2     A/Netherlands/10456/2024|A_/_H1N1|6B.1A.5a.2a....    NaN   E   E  NFSC   
3     A/Netherlands/10454/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E  NFSC   
4     A/Netherlands/11365/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E  NFSC   
...                                                 ...    ...  ..  ..   ...   
1473  A/Netherlands/10281/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E  NFSC   
1474  A/Netherlands/11508/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E  NFSC   
1475  A/Netherlands/10300/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E  NFSC   
1476  A/Netherlands/10275/2024|A_/_H1N1|6B.1A.5a.2a....    NaN   E   E  NFSC   
1477  A/Netherlands/10293/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E  NFSC   

       614   703           motif  
0   

Merge positions with nextclade

In [42]:
import pandas as pd

# # H1N1pdm HA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/ha/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/ha/positions_h1n1pdm_HA1.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/ha/glycosylation_sites_HA1.csv", sep=",")

# # H1N1pdm NA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/na/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/na/positions_h1n1pdm_NA.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/na/glycosylation_sites_NA.csv", sep=",")

# H1N1pdm PA data
df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/pa/nextclade.csv", sep=";")
df_positions = pd.read_csv("../../output/positions/h1n1pdm/pa/positions_h1n1pdm_PA.csv", sep=",")
df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/pa/glycosylation_sites_PA.csv", sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_positions = df_positions.reset_index(drop=True)
df_glycosylation = df_glycosylation.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

# Print the final dataframes to be used.
# print(f"This is 'df_nextclade' output: \n {df_nextclade.head()}")
# print(f"This is 'df_positions' output: \n {df_positions.head()}")
# print(f"This is 'df_glycosylation' output: \n {df_glycosylation.head()}")


# Merge all three tables into one Dataframe 'df'.
df = df_positions.merge(df_nextclade, how="left", on="seqName")


# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Reorder the DataFrame columns
df = df[final_columns]

print(df)


# HA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/ha/merged_tryout_pos_ha.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/na/merged_tryout_pos_na.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
df.to_csv("../../output/positions/h1n1pdm/pa/merged_tryout_pos_pa.csv", index=True)


                                                seqName  clade 198 199 motif
0     A/Netherlands/10452/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E    EE
1     A/Netherlands/00087/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E    EE
2     A/Netherlands/10456/2024|A_/_H1N1|6B.1A.5a.2a....    NaN   E   E    EE
3     A/Netherlands/10454/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E    EE
4     A/Netherlands/11365/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E    EE
...                                                 ...    ...  ..  ..   ...
1473  A/Netherlands/10281/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E    EE
1474  A/Netherlands/11508/2022|A_/_H1N1|6B.1A.5a.1|O...    NaN   E   E    EE
1475  A/Netherlands/10300/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E    EE
1476  A/Netherlands/10275/2024|A_/_H1N1|6B.1A.5a.2a....    NaN   E   E    EE
1477  A/Netherlands/10293/2024|A_/_H1N1|6B.1A.5a.2a|...    NaN   E   E    EE

[1478 rows x 5 columns]


Merge glycosylation with nextclade

In [ ]:
import pandas as pd



# # H1N1pdm HA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/ha/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/ha/positions_h1n1pdm_HA1.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/ha/glycosylation_sites_HA1.csv", sep=",")

# # H1N1pdm NA data
# df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/na/nextclade.csv", sep=";")
# df_positions = pd.read_csv("../../output/positions/h1n1pdm/na/positions_h1n1pdm_NA.csv", sep=",")
# df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/na/glycosylation_sites_NA.csv", sep=",")

# H1N1pdm PA data
df_nextclade = pd.read_csv("../../output/nextclade/flu/h1n1pdm/pa/nextclade.csv", sep=";")
df_positions = pd.read_csv("../../output/positions/h1n1pdm/pa/positions_h1n1pdm_PA.csv", sep=",")
df_glycosylation = pd.read_csv("../../output/glycosylation/flu/h1n1pdm/pa/glycosylation_sites_PA.csv", sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_positions = df_positions.reset_index(drop=True)
df_glycosylation = df_glycosylation.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

# Print the final dataframes to be used.
# print(f"This is 'df_nextclade' output: \n {df_nextclade.head()}")
# print(f"This is 'df_positions' output: \n {df_positions.head()}")
# print(f"This is 'df_glycosylation' output: \n {df_glycosylation.head()}")


# Merge all three tables into one Dataframe 'df'.
df = df_glycosylation.merge(df_nextclade, how="left", on="seqName")


# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Reorder the DataFrame columns
df = df[final_columns]

print(df)


# HA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/ha/merged_tryout_glyc_ha.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
# df.to_csv("../../output/positions/h1n1pdm/na/merged_tryout_glyc_na.csv", index=True)

# NA Save the results with clades showing all culumns and motif to CSV.
df.to_csv("../../output/positions/h1n1pdm/pa/merged_tryout_glyc_pa.csv", index=True)

Sankemake merge all

In [ ]:
import pandas as pd

#
# --------------------------   INPUT   ------------------------
#

# Set 'positions' to look for per lineage and segment using the input Excel file

nextclade_csv = str(snakemake.input.nextclade_csv)
positions_csv = str(snakemake.input.positions_csv)
glycosylation_csv = str(snakemake.input.glycosylation_csv)

#
# --------------------------   INPUT   ------------------------
#

merged_csv = str(snakemake.output.merged_csv)

#
# --------------------------   Prepare Dataframes   ------------------------
#

# H1N1pdm PA data
df_nextclade = pd.read_csv(nextclade_csv, sep=";")
df_positions = pd.read_csv(positions_csv, sep=",")
df_glycosylation = pd.read_csv(glycosylation_csv, sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_positions = df_positions.reset_index(drop=True)
df_glycosylation = df_glycosylation.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

#
# --------------------------  Merge Dataframes   ------------------------
#

# Merge all three tables into one Dataframe 'df'.
df = df_positions.merge(df_glycosylation, how="left", on="seqName") \
                .merge(df_nextclade, how="left", on="seqName")

#
# ------  Remove unwanted columns from nextclade after merge   -----------
#

# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

#
# ------  Sort and renaming Pos columns and adding 'motif' column  -----------
#

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

# Changin the order of columns
if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Store the reordered DataFrame columns to 'df'.
df = df[final_columns]

print(df)


# HA Save the results to CSV.
df.to_csv(merged_csv, index=True)


Snakemake merge_positions_nextclade_csv

In [ ]:
import pandas as pd

#
# --------------------------   INPUT   ------------------------
#

# Set 'positions' to look for per lineage and segment using the input Excel file

nextclade_csv = str(snakemake.input.nextclade_csv)
positions_csv = str(snakemake.input.positions_csv)

#
# --------------------------   INPUT   ------------------------
#

merged_csv = str(snakemake.output.merged_csv)

#
# --------------------------   Prepare Dataframes   ------------------------
#

# H1N1pdm PA data
df_nextclade = pd.read_csv(nextclade_csv, sep=";")
df_positions = pd.read_csv(positions_csv, sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_positions = df_positions.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

#
# --------------------------  Merge Dataframes   ------------------------
#

# Merge all three tables into one Dataframe 'df'.
df = df_positions.merge(df_nextclade, how="left", on="seqName")

#
# ------  Remove unwanted columns from nextclade after merge   -----------
#

# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

#
# ------  Sort and renaming Pos columns and adding 'motif' column  -----------
#

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

# Changin the order of columns
if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Store the reordered DataFrame columns to 'df'.
df = df[final_columns]

print(df)


# HA Save the results to CSV.
df.to_csv(merged_csv, index=True)


Snakemake merge_glycosylation_nextclade

In [ ]:
import pandas as pd

#
# --------------------------   INPUT   ------------------------
#

# Set 'positions' to look for per lineage and segment using the input Excel file

nextclade_csv = str(snakemake.input.nextclade_csv)
glycosylation_csv = str(snakemake.input.glycosylation_csv)

#
# --------------------------   INPUT   ------------------------
#

merged_csv = str(snakemake.output.merged_csv)

#
# --------------------------   Prepare Dataframes   ------------------------
#

# H1N1pdm PA data
df_nextclade = pd.read_csv(nextclade_csv, sep=";")
df_glycosylation = pd.read_csv(glycosylation_csv, sep=",")

# Reset the indexes for the dataframes
df_nextclade = df_nextclade.reset_index(drop=True)
df_glycosylation = df_glycosylation.reset_index(drop=True)

# Drop the surplus column "index" from the nextclade dataframe 
df_nextclade = df_nextclade.drop(["index"], axis=1)

#
# --------------------------  Merge Dataframes   ------------------------
#

# Merge all three tables into one Dataframe 'df'.
df = df_glycosylation.merge(df_nextclade, how="left", on="seqName")

#
# ------  Remove unwanted columns from nextclade after merge   -----------
#

# Check if "subclade" column exists, this is true for nextclade output for ha 
if "subclade" in df.columns:
    # Delete all columns to the right of "subclade", keeping columns "short-clade" and "subclade"
    cols_to_keep = df.columns[:df.columns.get_loc("subclade") + 1]
    df = df[cols_to_keep]
else:
    # Delete all columns to the right of "clade", in case that columns "short-clade" and "subclade" are non-exsisting.
    cols_to_keep = df.columns[:df.columns.get_loc("clade") + 1]
    df = df[cols_to_keep]

#
# ------  Sort and renaming Pos columns and adding 'motif' column  -----------
#

# Select columns with names that start with 'Pos '
pos_columns = [col for col in df.columns if col.startswith('Pos ')]

# Identify pos_columns that show variation in their values across the rows
variable_pos_columns = [col for col in pos_columns if df[col].nunique() > 1]

# Create the 'motif' column by concatenating values of the variable 'Pos ' columns
df['motif'] = df[variable_pos_columns].astype(str).agg(''.join, axis=1)

# Remove 'Pos ' part of the column names
df.rename(columns={col: col.replace('Pos ', '') for col in variable_pos_columns}, inplace=True)

# Sort the 'Pos ' columns by the numeric part
sorted_columns = sorted([col.replace('Pos ', '') for col in variable_pos_columns], key=int)

# Determine the final order of columns, considering the presence of optional columns
final_columns = ['seqName', 'clade'] + sorted_columns + ['motif']
if "short-clade" in df.columns:
    final_columns.append('short-clade')
if "subclade" in df.columns:
    final_columns.append('subclade')

# Changin the order of columns
if "subclade" in df.columns:
    final_columns = ['seqName', 'clade', 'short-clade', 'subclade'] + sorted_columns + ['motif']

# Store the reordered DataFrame columns to 'df'.
df = df[final_columns]

print(df)


# HA Save the results to CSV.
df.to_csv(merged_csv, index=True)

Merge the output from rule "merge_positions_nextclade" and rule "merge_glycosylation_nextclade"

In [14]:
import pandas as pd

#
# --------------------------   INPUT   ------------------------
#
 
positions_merged_csv = "../../output/positions/h1n1pdm/ha/positions_nextclade_h1n1pdm_HA1.csv"
glycosylation_merged_csv = "../../output/glycosylation/h1n1pdm/ha/glycosylation_nextclade_h1n1pdm_HA1.csv"

#
# --------------------------   OUTPUT   ------------------------
#

merged_csv = "../../output/positions/h1n1pdm/ha/positions_nextclade_glycosylation_h1n1pdm_HA1.csv"

#
# --------------------------   Prepare Dataframes   ------------------------
#

df_positions_merged = pd.read_csv(positions_merged_csv)
df_glycosylation_merged = pd.read_csv(glycosylation_merged_csv)


df = df_positions_merged.merge(df_glycosylation_merged, on="seqName", suffixes=('_pos', '_gly'))

# Remove columns that are not of interest.
if "subclade_gly" in df.columns:
    df = df.drop("subclade_gly", axis=1)
if "short-clade_gly" in df.columns:
    df = df.drop("short-clade_gly", axis=1)
if "clade_gly" in df.columns:
    df = df.drop("clade_gly", axis=1)
if "Unnamed: 0_gly" in df.columns:
    df = df.drop("Unnamed: 0_gly", axis=1)
if "Unnamed: 0_pos" in df.columns:
    df = df.drop("Unnamed: 0_pos", axis=1)


# Renaming columns
if "clade_pos" in df.columns:
    df = df.rename(columns={"clade_pos": "clade"})
if "short-clade_pos" in df.columns:
    df = df.rename(columns={"short-clade_pos": "short-clade"})  
if "subclade_pos" in df.columns:
    df = df.rename(columns={"subclade_pos" : "subclade"})




print(df.head())

#
# --------------------------   Save Dataframes   ------------------------
#

# Save the results to CSV.
df.to_csv(merged_csv, index=True)


                                             seqName        clade short-clade  \
0  A/Netherlands/11365/2022|A_/_H1N1|6B.1A.5a.1|O...   6B.1A.5a.1        5a.1   
1  A/Netherlands/10452/2024|A_/_H1N1|6B.1A.5a.2a|...  6B.1A.5a.2a       5a.2a   
2  A/Netherlands/00068/2022|A_/_H1N1|6B.1A.5a.1|O...   6B.1A.5a.1        5a.1   
3  A/Netherlands/00087/2022|A_/_H1N1|6B.1A.5a.1|O...   6B.1A.5a.1        5a.1   
4  A/Netherlands/10487/2024|A_/_H1N1|6B.1A.5a.2a|...  6B.1A.5a.2a       5a.2a   

  subclade 129 130 132 133 134 137  ... 190 216 222 224           motif_pos  \
0        B   D   K   V   T   A   P  ...   S   T   D   E  DKVTAPAKVGNTAASTDE   
1    C.1.9   D   N   V   T   A   P  ...   S   T   D   A  DNVTAPAKVGKTTDSTDA   
2        B   D   K   V   T   A   P  ...   S   T   D   E  DKVTAPAKVGNTAASTDE   
3        B   D   K   V   T   A   P  ...   S   T   D   E  DKVTAPAKVGNTAASTDE   
4    C.1.9   D   N   V   T   A   P  ...   S   T   D   A  DNVTAPAKVGKTTDSTDA   

     87   162   276   287         moti